<a href="https://colab.research.google.com/github/imiled/DL_Tools_For_Finance/blob/master/Main_RNN_learning_to_add_as_transaltion_task_colab_student_version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# An implementation of sequence to sequence learning to perform addition

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


## Sequence to sequence models

RNNs can be used for all sort of tasks, as illustrated in the image bellow:

![](http://cntk.ai/jup/paradigms.jpg)

Among these, one of the most prominent uses of RNNs are the Sequence to Sequence models, where one *translates* a sequence into another.

![](https://jeddy92.github.io/images/ts_intro/seq2seq_ts.png)

It can be used for time series multi-step prediction, as above, or for several #NLProc tasks:

* Question answering:

![](https://static.wixstatic.com/media/bede95_222a9958cbc94b8a800ee13486dadd7b~mv2.png)

* Language translation:

![](http://cntk.ai/jup/s2s.png)

* Image captioning:

![](https://i.pinimg.com/originals/0f/72/68/0f7268c8e7a59adca8c0cba848891a76.png)




## Encoder-Decoder LSTM

An encoder-decoder LSTM is a model comprised of two sub-models: one called the encoder that reads the input sequences and compresses it to a fixed-length internal representation, and an output model called the decoder that interprets the internal representation and uses it to predict the output sequence.


**Example:**
<br></br>
Given the input: "535+61", the network should predict the ouput: "596" character by character.
Padding is handled by using a repeated sentinel character (space).

Input may optionally be reversed, shown to increase performance in many tasks in
[Learning to Execute](http://arxiv.org/abs/1410.4615)
and
[Sequence to Sequence Learning with Neural Networks](http://papers.nips.cc/paper/5346-sequence-to-sequence-learning-with-neural-networks.pdf). Theoretically it introduces shorter term dependencies between source and target.



Below you can find some expected results.

Two digits reversed:
+ One layer LSTM (128 HN), 5k training examples = 99% train/test accuracy in 55 epochs

Three digits reversed:
+ One layer LSTM (128 HN), 50k training examples = 99% train/test accuracy in 100 epochs

Four digits reversed:
+ One layer LSTM (128 HN), 400k training examples = 99% train/test accuracy in 20 epochs

Five digits reversed:
+ One layer LSTM (128 HN), 550k training examples = 99% train/test accuracy in 30 epochs

In [ ]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [ ]:
try:
    from tensorflow.python.util import module_wrapper as deprecation
except ImportError:
    from tensorflow.python.util import deprecation_wrapper as deprecation
deprecation._PER_MODULE_WARNING_LIMIT = 0

In [ ]:
import numpy as np
from six.moves import range
import sys
import tensorflow as tf

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

## 1-hot character encoder for sequences

In [ ]:
class CharacterTable(object):
    """Given a set of characters:
    + Encode them to a one-hot integer representation
    + Decode the one-hot or integer representation to their character output
    + Decode a vector of probabilities to their character output
    """
    def __init__(self, chars):
        """Initialize character table.

        # Arguments
            chars: Characters that can appear in the input.
        """
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))

    def encode(self, C, num_rows):
        """One-hot encode given string C.

        # Arguments
            C: string, to be encoded.
            num_rows: Number of rows in the returned one-hot encoding. This is
                used to keep the # of rows for each data the same.
        """
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x

    def decode(self, x, calc_argmax=True):
        """Decode the given vector or 2D array to their character output.

        # Arguments
            x: A vector or a 2D array of probabilities or one-hot representations;
                or a vector of character indices (used with `calc_argmax=False`).
            calc_argmax: Whether to find the character index with maximum
                probability, defaults to `True`.
        """
        if calc_argmax:
            x = x.argmax(axis=-1)
        return ''.join(self.indices_char[x] for x in x)

In [ ]:
# All the numbers, plus sign and space for padding.
chars = '0123456789+ '
ctable = CharacterTable(chars)

In [ ]:
# try encoding something
str_ = '0 +'
encoded=ctable.encode(str_,num_rows=6)

In [ ]:
# now, decode it!
ctable.decode(encoded)

'0 +   '

In [ ]:
# Define colors to pretty print our results
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

## Generate training sequences


In [ ]:
def generate_training(TRAINING_SIZE = 50000, DIGITS = 3, REVERSE = True):

    # Maximum length of input is 'int + int' (e.g., '345+678'). Maximum length of
    # int is DIGITS.
    MAXLEN = DIGITS + 1 + DIGITS

    questions = []
    answers = []
    seen = set()    #enleve les dupliquer
    print('Generating data...')
    while len(questions) < TRAINING_SIZE:
        f = lambda: int(''.join(np.random.choice(list('0123456789'))
                        for i in range(np.random.randint(1, DIGITS + 1))))
        a, b = f(), f()
        # Skip any addition questions we've already seen
        # Also skip any such that x+Y == Y+x (hence the sorting).
        key = tuple(sorted((a, b))) #ordenar los datos
        if key in seen:
            continue
        seen.add(key)
        # Pad the data with spaces such that it is always MAXLEN.
        #padding=pour que les sequences ont la meme taille
        q = '{}+{}'.format(a, b)
        query = q + ' ' * (MAXLEN - len(q))
        ans = str(a + b)
        # Answers can be of maximum size DIGITS + 1.
        ans += ' ' * (DIGITS + 1 - len(ans))
        if REVERSE:
            # Reverse the query, e.g., '12+345  ' becomes '  543+21'. (Note the
            # space used for padding.)
            query = query[::-1]
        questions.append(query)
        answers.append(ans)
    print('Total addition questions:', len(questions))
    out = np.vstack((np.array(questions), np.array(answers)))
    return out.T

In [ ]:
train3char = generate_training(TRAINING_SIZE=50000, DIGITS=3, REVERSE=False)

Generating data...
Total addition questions: 50000


In [ ]:
train3char.shape

(50000, 2)

In [ ]:
# let's see some samples
r = np.random.randint(low=0, high=train3char.shape[0], size=4)
train3char[r, :]

array([['501+21 ', '522 '],
       ['563+908', '1471'],
       ['393+57 ', '450 '],
       ['680+10 ', '690 ']], dtype='<U7')

In [ ]:
ctable.encode('99',2).shape

(2, 12)

In [ ]:
ctable

[' ', '+', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

### Vectorize training sample with the character encoder

In [ ]:
def vectorization(train, DIGITS, voc_len, encoder):
    """
    In this method we use the encoding table to convert characters to vectors.
    """
    MAXLEN = 2*DIGITS+1
    print('Vectorization...')
    x = np.zeros(shape=(train.shape[0], MAXLEN, voc_len), dtype=np.float)
    y = np.zeros(shape=(train.shape[0],DIGITS+1,  voc_len), dtype=np.float)
    # encode questions
    for i, sentence in enumerate(train[:, 0]):
        x[i] = encoder.encode(sentence, MAXLEN)
    # encode answers
    for i, sentence in enumerate(train[:, 1]):
        y[i] =  encoder.encode(sentence, DIGITS+1)

    # Shuffle (x, y) in unison as the later parts of x will almost all be larger
    # digits.
    indices = np.arange(len(y))
    np.random.shuffle(indices)
    x = x[indices]
    y = y[indices]

    # Explicitly set apart 10% for validation data that we never train over.
    split_at = len(x) - len(x) // 10
    (x_train, x_val) = x[:split_at], x[split_at:]
    (y_train, y_val) = y[:split_at], y[split_at:]

    print('Shapes in training Data:')
    print(x_train.shape)
    print(y_train.shape)

    print('Shapes in validation Data:')
    print(x_val.shape)
    print(y_val.shape)
    return x_train, y_train, x_val, y_val

In [ ]:
x_train_3char, y_train_3char, x_val_3char, y_val_3char = vectorization(train3char, DIGITS=3, voc_len=len(chars), encoder=ctable)

Vectorization...
Shapes in training Data:
(45000, 7, 12)
(45000, 4, 12)
Shapes in validation Data:
(5000, 7, 12)
(5000, 4, 12)


In [ ]:
x_train_3char[0]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

## Model

### Model definition

The encoder consists of a single RNN that compress the input sequence into a single vector.

Then, we repeat this vector to feed it to the the decoding RNN.
Finally, we use a Dense layer (an embedding, shared across all output timestamps) to produce the output characters. 

In [ ]:
from tensorflow.keras.layers import GRU, LSTM, RNN,RepeatVector, TimeDistributed, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop

In [ ]:
def build_model(chars, rnn_type='gru', DIGITS=3, HIDDEN_SIZE=10,  DECODER_LAYERS=1):
    tf.keras.backend.clear_session()
    MAXLEN = 2*DIGITS + 1
    if rnn_type.lower() == 'gru':
        rnn = GRU
    elif rnn_type.lower() == 'lstm':
        rnn = LSTM
    elif rnn_type.lower() == 'rnn':
        rnn = RNN
    else:
        print('{rnn_type} RNN type not covered'.format(rnn_type=rnn_type))
        sys.exit(0)
        
    print('Build model...')
    model = Sequential()
    model.add(rnn(units=HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))
    model.add(RepeatVector(DIGITS+1))

    # "Encode" the input sequence using an RNN, producing an output of HIDDEN_SIZE.
    # Note: In a situation where your input sequences have a variable length,
    # use input_shape=(None, num_feature).
    
    # As the decoder RNN's input, repeatedly provide with the last output of
    # RNN for each time step. Repeat 'DIGITS + 1' times as that's the maximum
    # length of output, e.g., when DIGITS=3, max output is 999+999=1998.
    # for this, use layers.RepeatVector()
  
    
    # The decoder RNN could be multiple layers stacked or a single layer.
    for _ in range(DECODER_LAYERS):
        # By setting return_sequences to True, return not only the last output but
        # all the outputs so far in the form of (num_samples, timesteps,
        # output_dim). This is necessary as TimeDistributed in the below expects
        # the first dimension to be the timesteps.
          model.add(rnn(HIDDEN_SIZE, return_sequences=True))
 

    # Apply a dense layer to the every temporal slice of an input. For each of step
    # of the output sequence, decide which character should be chosen.
    dense=Dense(units=len(chars), activation='softmax')
    model.add(TimeDistributed(dense))
    model.compile(loss='categorical_crossentropy',
                  optimizer=RMSprop(learning_rate=0.1, clip=1),
                  metrics=['categorical_accuracy'])
    model.summary()
    return model

In [ ]:
model3char_gru_10 = build_model(chars, rnn_type='gru', DIGITS=3, HIDDEN_SIZE=10, DECODER_LAYERS=1)
model3char_gru_20 = build_model(chars, rnn_type='gru', DIGITS=3, HIDDEN_SIZE=20, DECODER_LAYERS=1)
model3char_lstm_10 = build_model(chars, rnn_type='lstm', DIGITS=3, HIDDEN_SIZE=10, DECODER_LAYERS=1)
model3char_lstm_20 = build_model(chars, rnn_type='lstm', DIGITS=3, HIDDEN_SIZE=20, DECODER_LAYERS=1)

model3char_gru_10_dec10 = build_model(chars, rnn_type='gru', DIGITS=3, HIDDEN_SIZE=10, DECODER_LAYERS=10)
model3char_gru_40_dec10 = build_model(chars, rnn_type='gru', DIGITS=3, HIDDEN_SIZE=40, DECODER_LAYERS=10)
model3char_lstm_10_dec4 = build_model(chars, rnn_type='lstm', DIGITS=3, HIDDEN_SIZE=10, DECODER_LAYERS=4)
model3char_lstm_20_dec4 = build_model(chars, rnn_type='lstm', DIGITS=3, HIDDEN_SIZE=20, DECODER_LAYERS=4)

Build model...
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru (GRU)                    (None, 10)                720       
_________________________________________________________________
repeat_vector (RepeatVector) (None, 4, 10)             0         
_________________________________________________________________
gru_1 (GRU)                  (None, 4, 10)             660       
_________________________________________________________________
time_distributed (TimeDistri (None, 4, 12)             132       
Total params: 1,512
Trainable params: 1,512
Non-trainable params: 0
_________________________________________________________________
Build model...
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru (GRU)                    (None, 20)                2040      
____

In [ ]:
#comparar con el time sereie

### Model training

In each epoch, print how many characters are we guessing correctly

In [ ]:
def train(model, x_train, y_train, x_val, y_val,  n_epochs=20, BATCH_SIZE=128, REVERSE=False):
    # Train the model each generation and show predictions against the validation
    # dataset.
    for iteration in range(1, n_epochs+1):
        print()
        print('-' * 50)
        print('Iteration', iteration)
        # fit, providing the validation data that we have created earlier
        model.fit(x_train,y_train,epochs=n_epochs, batch_size=BATCH_SIZE, validation_data=[x_val, y_val])
        
        # Select N samples from the validation set at random so we can visualize
        # errors.
        N=5
        for i in range(N):
            ind = np.random.randint(0, len(x_val))
            rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
            q = ctable.decode(rowx[0])
            correct = ctable.decode(rowy[0])

            preds = model.predict_classes(rowx, batch_size=1, verbose=1)
            guess = ctable.decode(preds[0], calc_argmax=False)
            
            print('Question: ', q[::-1] if REVERSE else q, end=' ')
            print('Answer: ', correct, end=' ')
            if correct == guess:
                print(colors.ok + '☑' + colors.close, end=' ')
            else:
                print(colors.fail + '☒' + colors.close, end=' ')
            print('Guess: ', guess, end='\n')

In [ ]:
  model3char.fit(x_train_3char,y_train_3char,epochs=1, batch_size=128, validation_data=[x_val_3char, y_val_3char])

In [ ]:
# train 
x=train(model3char_lstm_10_dec4, x_train_3char, y_train_3char, x_val_3char, y_val_3char,  n_epochs=15, BATCH_SIZE=128, REVERSE=False)
    


--------------------------------------------------
Iteration 1
Train on 45000 samples, validate on 5000 samples
Epoch 1/15
45000/45000 [==============================] - 6s 125us/sample - loss: 1.0533 - categorical_accuracy: 0.5857 - val_loss: 1.1027 - val_categorical_accuracy: 0.5616
Epoch 2/15
45000/45000 [==============================] - 6s 125us/sample - loss: 1.0501 - categorical_accuracy: 0.5863 - val_loss: 1.0039 - val_categorical_accuracy: 0.6054
Epoch 3/15
45000/45000 [==============================] - 6s 124us/sample - loss: 1.0439 - categorical_accuracy: 0.5886 - val_loss: 1.0514 - val_categorical_accuracy: 0.5840
Epoch 4/15
45000/45000 [==============================] - 6s 123us/sample - loss: 1.0411 - categorical_accuracy: 0.5892 - val_loss: 1.0175 - val_categorical_accuracy: 0.5993
Epoch 5/15
45000/45000 [==============================] - 6s 126us/sample - loss: 1.0379 - categorical_accuracy: 0.5913 - val_loss: 1.1190 - val_categorical_accuracy: 0.5728
Epoch 6/15
45000/

In [ ]:
ctable.decode(x_val_3char[10]),ctable.decode(y_val_3char[10])

('475+503', '978 ')

## Do the same with 5 digits sums.

* Generate train and validation sets
* Vectorize the datasets
* Build the model for handling 5 digit sequences
* Train it

In [ ]:
# generate train
train5char = ?

In [ ]:
r = np.random.randint(low=0, high=train5char.shape[0], size=4)
train5char[r, :]

In [ ]:
# vectorize
x_train_5char, y_train_5char, x_val_5char, y_val_5char = ?

In [ ]:
# build the model
model5char = ?

In [ ]:
# train
?

### Check Five digits reversed:
+ With one LSTM layer (128 HN) and 550k training examples, you should be able to reach 99% train/test accuracy within 30 epochs


In [ ]:
?

## Next steps

Wanna here more about Seq2Seq models? There are several resources that might interest you:

* [Keras intro](https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html) to Seq2Seq with the functional API. Here, you will learn how to feed the state vectors and 1-char target sequence to the decoder to produce predictions for the next character. I recommend you to try this approach in the weather time-series dataset =)

* Checkout this [excellent post](https://machinelearningmastery.com/how-to-develop-lstm-models-for-multi-step-time-series-forecasting-of-household-power-consumption/) where Seq2Seq models are used for time-series forecasting of household power consumption. 

  There, you will not only use LSTM encoders/decorders, but also combine one-dimensional CNN with LSTMs. In the latter case, the CNN is used as an encoder to learn features from sub-sequences of input data which are provided as time steps to a decoding LSTM. This architecture is called a [CNN-LSTM](https://arxiv.org/abs/1411.4389).

  We can go even further and use the CNN in combination with the LSTM as our encoder. This approach is called [ConvLSTM](https://arxiv.org/abs/1506.04214v1) and is expected to capture better spatiotemporal correlations. 

* An important improvement to RNNs is the use of the [Attention mechanism](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html), that helps memorizing long source sentences. Attention models are not extensively covered in the master (a couple of hours at the very end of the DL module), so that's the kind of material you could explore in your Master's Thesis ;) The cenit of the attention mechanisms is the so called [*self-attention*](http://papers.nips.cc/paper/7181-attention-is-all-you-need.pdf), which has given rise to the Transformer arquitecture. Based upon it, OpenAI has been capable of generating text of [very high quality](https://talktotransformer.com/)!

* In the 2nd module, you will see how to apply these networks to NLP problems. Meanwhile, you might want to read about [image captioning](https://www.tensorflow.org/tutorials/text/image_captioning), [neural translation with attention](https://www.tensorflow.org/tutorials/text/nmt_with_attention), [text generation](https://www.tensorflow.org/tutorials/text/text_generation) or [question-answer generation]()
